# TP03 - Classificação
## Dataset: Customers Data (Horeca vs Retail)

Análise de gastos anuais em diferentes categorias de produtos para classificação de canais de venda.

### 1. Importação de bibliotecas e carregamento dos dados

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

In [2]:
url = 'https://raw.githubusercontent.com/cassiusf/datasets/main/customers_data.csv'
df = pd.read_csv(url)

print(f"Shape do dataset: {df.shape}")
print(f"\nPrimeiras linhas:")
df.head()

Shape do dataset: (440, 8)

Primeiras linhas:


,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
0,2,3,12669,9656,7561,214,2674,1338
1,2,3,7057,9810,9568,1762,3293,1776
2,2,3,6353,8808,7684,2405,3516,7844
3,1,3,13265,1196,4221,6404,507,1788
4,2,3,22615,5410,7198,3915,1777,5185


### 2. Análise exploratória inicial

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440 entries, 0 to 439
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   Channel           440 non-null    int64
 1   Region            440 non-null    int64
 2   Fresh             440 non-null    int64
 3   Milk              440 non-null    int64
 4   Grocery           440 non-null    int64
 5   Frozen            440 non-null    int64
 6   Detergents_Paper  440 non-null    int64
 7   Delicassen        440 non-null    int64
dtypes: int64(8)
memory usage: 27.6 KB


In [4]:
print("Estatísticas descritivas:")
df.describe()

Estatísticas descritivas:


,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
count,440.000000,440.000000,440.000000,440.000000,440.000000,440.000000,440.000000,440.000000
mean,1.322727,2.543182,12000.297727,5796.265909,7951.277273,3071.931818,2881.493182,1524.870455
std,0.468052,0.774272,12647.328865,7380.377175,9503.162829,4854.673333,4767.854448,2820.105937
min,1.000000,1.000000,3.000000,55.000000,3.000000,25.000000,3.000000,3.000000
25%,1.000000,2.000000,3127.750000,1533.000000,2153.000000,742.250000,256.750000,408.250000
50%,1.000000,3.000000,8504.000000,3627.000000,4755.500000,1526.000000,816.500000,965.500000
75%,2.000000,3.000000,16933.750000,7190.250000,10655.750000,3554.250000,3922.000000,1820.250000
max,2.000000,3.000000,112151.000000,73498.000000,92780.000000,60869.000000,40827.000000,47943.000000


### 3. Tratamento de dados

In [5]:
print(f"Dados ausentes por coluna:")
print(df.isnull().sum())

rows_before = len(df)
df = df.dropna()
rows_after = len(df)

print(f"\nLinhas removidas: {rows_before - rows_after}")
print(f"Shape após remoção: {df.shape}")

Dados ausentes por coluna:
Channel             0
Region              0
Fresh               0
Milk                0
Grocery             0
Frozen              0
Detergents_Paper    0
Delicassen          0
dtype: int64

Linhas removidas: 0
Shape após remoção: (440, 8)


In [6]:
print(f"Valores únicos em Channel antes: {df['Channel'].unique()}")

df['Channel'] = df['Channel'].map({1: 1, 2: 0})

print(f"Valores únicos em Channel depois: {df['Channel'].unique()}")
print(f"\nDistribuição da variável alvo:")
print(df['Channel'].value_counts())
print(f"\nProporção:")
print(df['Channel'].value_counts(normalize=True))

Valores únicos em Channel antes: [2 1]
Valores únicos em Channel depois: [0 1]

Distribuição da variável alvo:
Channel
1    298
0    142
Name: count, dtype: int64

Proporção:
Channel
1    0.677273
0    0.322727
Name: proportion, dtype: float64


### 4. Separação em treino e teste

In [7]:
X = df.drop(['Channel'], axis=1)
y = df['Channel']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

print(f"Tamanho do conjunto de treino: {X_train.shape}")
print(f"Tamanho do conjunto de teste: {X_test.shape}")
print(f"\nDistribuição da variável alvo no treino:")
print(y_train.value_counts(normalize=True))
print(f"\nDistribuição da variável alvo no teste:")
print(y_test.value_counts(normalize=True))

Tamanho do conjunto de treino: (330, 7)
Tamanho do conjunto de teste: (110, 7)

Distribuição da variável alvo no treino:
Channel
1    0.675758
0    0.324242
Name: proportion, dtype: float64

Distribuição da variável alvo no teste:
Channel
1    0.681818
0    0.318182
Name: proportion, dtype: float64


### 5. Modelo Random Forest

In [8]:
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

y_pred_rf = rf_model.predict(X_test)

print("Random Forest - Métricas:")
print(f"Acurácia: {accuracy_score(y_test, y_pred_rf):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_rf):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_rf):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred_rf):.4f}")
print(f"\nMatriz de Confusão:")
print(confusion_matrix(y_test, y_pred_rf))

Random Forest - Métricas:
Acurácia: 0.9091
Precision: 0.9333
Recall: 0.9333
F1-Score: 0.9333

Matriz de Confusão:
[[30  5]
 [ 5 70]]


### 6. Modelo XGBoost

In [9]:
xgb_model = xgb.XGBClassifier(random_state=42)
xgb_model.fit(X_train, y_train)

y_pred_xgb = xgb_model.predict(X_test)

print("XGBoost - Métricas:")
print(f"Acurácia: {accuracy_score(y_test, y_pred_xgb):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_xgb):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_xgb):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred_xgb):.4f}")
print(f"\nMatriz de Confusão:")
print(confusion_matrix(y_test, y_pred_xgb))

XGBoost - Métricas:
Acurácia: 0.9000
Precision: 0.9211
Recall: 0.9333
F1-Score: 0.9272

Matriz de Confusão:
[[29  6]
 [ 5 70]]


### 7. Comparação dos modelos

In [10]:
comparacao = pd.DataFrame({
    'Modelo': ['Random Forest', 'XGBoost'],
    'Acurácia': [accuracy_score(y_test, y_pred_rf), accuracy_score(y_test, y_pred_xgb)],
    'Precision': [precision_score(y_test, y_pred_rf), precision_score(y_test, y_pred_xgb)],
    'Recall': [recall_score(y_test, y_pred_rf), recall_score(y_test, y_pred_xgb)],
    'F1-Score': [f1_score(y_test, y_pred_rf), f1_score(y_test, y_pred_xgb)]
})

print("Comparação dos modelos:")
comparacao

Comparação dos modelos:


,Modelo,Acurácia,Precision,Recall,F1-Score
0,Random Forest,0.909091,0.933333,0.933333,0.933333
1,XGBoost,0.900000,0.921053,0.933333,0.927152


### 8. Conclusão

**Modelo escolhido: Random Forest**

Com base nos resultados obtidos, o Random Forest apresentou melhor desempenho:
- **Random Forest**: Acurácia 90.91%, F1-Score 93.33%
- **XGBoost**: Acurácia 90.00%, F1-Score 92.72%

O Random Forest obteve resultados superiores em todas as métricas, com destaque para o F1-Score (93.33% vs 92.72%), que representa o equilíbrio entre precision e recall. A diferença de 0.91% na acurácia e 0.61% no F1-Score favorece o Random Forest para esta tarefa de classificação entre canais Horeca e Retail.